In [4]:
import pandas as pd
import jieba
import time
import math
import re
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata
from openai import OpenAI

# ==================== 0. 安裝與設定中文字型 ====================
if not os.path.exists('SimHei.ttf'):
    print("正在下載中文字型 (SimHei)...")
    os.system('wget -q https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf')

font_path = 'SimHei.ttf'
my_font = fm.FontProperties(fname=font_path)
print("✅ 中文字型準備完成")

# ==================== 1. 環境與 AI 函式設置 (整合 modern_methods.ipynb) ====================

# 1. API Client 初始化
# 從 Colab Secrets 讀取金鑰，並初始化 OpenAI Client
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()
print("✅ OpenAI Client 初始化完成")

# Part A 測試數據 (來自原始程式碼)
documents_for_A1 = [
    "人工智慧正在改變世界，機器學習是其核心技術",
    "深度學習推動了人工智慧的發展，特別是在圖像識別領域",
    "今天天氣很好，適合出去運動",
    "機器學習和深度學習都是人工智慧的重要分支",
    "運動有益健康，每天都應該保持運動習慣",
]

# 2. 真實 AI 函式定義 (來自 modern_methods.ipynb)
# B-1: 語意相似度計算
def ai_similarity(text1, text2, api_key=None):
    prompt = f"""
請評估以下兩段文字的語意相似度。
文字1: {text1}
文字2: {text2}
請只回答一個0-100的數字，代表相似度百分比。
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是繁體中文語意分析專家，只能回答數字。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=10
        )
        result_text = response.choices[0].message.content.strip()
        match = re.search(r'\d+', result_text)
        if match:
            score = int(match.group())
            return max(0, min(100, score))
        return 0
    except Exception as e:
        # print(f"API 呼叫發生錯誤: {e}") # 執行時可取消註解查看錯誤
        return 0

# B-2: AI 文本分類
def ai_classify(text, api_key=None):
    prompt = f"""
請分析以下文本的情感與主題。
文本內容: "{text}"
請回傳一個 JSON 物件，包含以下欄位：
1. "sentiment": 判斷情感，只能是 "正面"、"負面" 或 "中性"。
2. "topic": 判斷主題類別 (例如：美食、科技、運動、娛樂、旅遊、生活...等，請依據內容自動判斷最適合的詞)。
3. "confidence": 信心分數 (0.0 到 1.0 之間)。
只回傳 JSON 字串，不要有其他說明。
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個精準的文本分析 API，只回傳 JSON 格式。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=100
        )
        result_text = response.choices[0].message.content.strip()
        if result_text.startswith("```json"):
            result_text = result_text.replace("```json", "").replace("```", "")
        data = json.loads(result_text)
        return data
    except Exception as e:
        # print(f"AI 分類發生錯誤: {e}") # 執行時可取消註解查看錯誤
        return {"sentiment": "未知", "topic": "未知", "confidence": 0.0}

# B-3: AI 自動摘要
def ai_summarize(text, ratio=0.3, api_key=None):
    target_percent = int(ratio * 100)
    prompt = f"""
請扮演一位專業編輯，將以下文章進行摘要。
要求：
1. 保留文章的核心資訊與關鍵論點。
2. 語句必須通順流暢。
3. 摘要長度大約控制在原文的 {target_percent}% 左右。
文章內容:
{text}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一位精通繁體中文的專業編輯。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=300
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        # print(f"AI 摘要發生錯誤: {e}") # 執行時可取消註解查看錯誤
        return "摘要生成失敗"

# ==================== 2. 定義 Part A 傳統方法類別 (修正 RuleBasedClassifier) ====================

class RuleBasedClassifier:
    """
    修正後的規則分類器：
    邏輯完全比照 traditional_methods.ipynb 的 TopicClassifier 與 RuleBasedSentimentClassifier
    """
    def __init__(self):
        # 1. 主題關鍵字
        self.topic_keywords = {
            '科技': ['AI', '人工智慧', '電腦', '軟體', '程式', '演算法'],
            '運動': ['運動', '健身', '跑步', '游泳', '球類', '比賽'],
            '美食': ['吃', '食物', '餐廳', '美味', '料理', '烹飪'],
            '旅遊': ['旅行', '景點', '飯店', '機票', '觀光', '度假']
        }

        # 2. 情感關鍵字與程度副詞
        self.positive_words = ['好', '棒', '優秀', '喜歡', '推薦', '滿意', '開心', '值得', '精彩', '完美']
        self.negative_words = ['差', '糟', '失望', '討厭', '不推薦', '浪費', '無聊', '爛', '糟糕', '差勁']
        self.negation_words = ['不', '沒', '無', '非', '別']

        self.degree_words = {
            '很': 1.5, '非常': 2.0, '超': 2.0, '太': 1.8, '特別': 1.5,
            '真的': 1.3, '真': 1.3, '十分': 1.8, '相當': 1.5
        }

    def classify(self, text):
        # ========== Part 1: 主題分類 (Topic) ==========
        # 邏輯：使用 count() 計算出現次數，信心度為 raw score (整數)
        topic_scores = {topic: 0 for topic in self.topic_keywords}

        for topic, keywords in self.topic_keywords.items():
            for kw in keywords:
                # 差異點：使用 text.count(kw) 而非 if kw in text
                count = text.count(kw)
                topic_scores[topic] += count

        max_score = max(topic_scores.values())
        if max_score == 0:
            best_topic = "未分類"
            topic_confidence = 0
        else:
            best_topic = max(topic_scores, key=topic_scores.get)
            topic_confidence = max_score  # 直接使用得分作為信心度

        # ========== Part 2: 情感分類 (Sentiment) ==========
        # 邏輯：加入程度副詞加權，信心度為正負分數差值
        positive_count = 0
        negative_count = 0

        i = 0
        while i < len(text):
            # 檢查程度副詞
            degree_multiplier = 1.0
            for degree_word, weight in self.degree_words.items():
                if text[i:i+len(degree_word)] == degree_word:
                    degree_multiplier = weight
                    i += len(degree_word)
                    break

            # 檢查否定詞
            is_negated = False
            for neg_word in self.negation_words:
                if text[i:i+len(neg_word)] == neg_word:
                    is_negated = True
                    i += len(neg_word)
                    break

            # 檢查正面詞
            found = False
            for pos_word in self.positive_words:
                if text[i:i+len(pos_word)] == pos_word:
                    if is_negated:
                        negative_count += degree_multiplier
                    else:
                        positive_count += degree_multiplier
                    i += len(pos_word)
                    found = True
                    break
            if found: continue

            # 檢查負面詞
            for neg_word in self.negative_words:
                if text[i:i+len(neg_word)] == neg_word:
                    if is_negated:
                        positive_count += degree_multiplier
                    else:
                        negative_count += degree_multiplier
                    i += len(neg_word)
                    found = True
                    break

            if not found:
                i += 1

        if positive_count > negative_count:
            sentiment = '正面'
        elif negative_count > positive_count:
            sentiment = '負面'
        else:
            sentiment = '中性'

        # 這裡的 confidence 回傳的是「主題分類的信心度」，
        # 因為比較報告主要是在比較主題分類的準確率。
        return {
            "topic": best_topic,
            "sentiment": sentiment,
            "raw_score": topic_confidence  # 例如：2 (代表匹配到2次)
        }

# 保留 StatisticalSummarizer 和 TFIDFSimilarity_For_Comparison 的原始定義


class StatisticalSummarizer:
    # ... (保留 StatisticalSummarizer 的原始定義)
    def __init__(self):
        # 擴充停用詞以改善效果
        self.stop_words = set(['的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都', '一', '一個', '上', '也', '很', '到', '說', '要', '去', '你', '這', '那', '會', '為', '著', '之', '與', '及', '其', '或', '但', '而'])

    def sentence_score(self, sentence, word_freq, index=0, total_sentences=1):
        # (保留 sentence_score 函式原始邏輯)
        if not sentence: return 0
        score = sum(word_freq.get(w, 0) for w in sentence if w not in self.stop_words)
        if index == 0: score *= 2.0
        elif index == total_sentences - 1: score *= 1.5
        if len(sentence) < 5: score *= 0.5 # 長度懲罰
        return score

    def summarize(self, text, ratio=0.3):
        # (保留 summarize 函式原始邏輯)
        sentences = re.split(r'([。！？])', text)
        clean_sentences = []
        for i in range(0, len(sentences)-1, 2):
            if sentences[i].strip():
                clean_sentences.append(sentences[i] + sentences[i+1])
        if len(sentences) % 2 != 0 and sentences[-1].strip(): # 處理最後可能剩餘的部分
             clean_sentences.append(sentences[-1])
        if not clean_sentences: return "", []

        all_content = "".join(clean_sentences)
        # 簡單分詞：這裡以「字元」為單位，過濾停用詞和標點
        all_words = [c for c in all_content if c not in self.stop_words and c.strip() and c not in '。！？，、：；「」『』']
        word_freq = Counter(all_words)
        total_words = len(all_words)
        word_freq_norm = {k: v/total_words for k, v in word_freq.items()}

        sentence_scores = []
        for i, sent in enumerate(clean_sentences):
            score = self.sentence_score(sent, word_freq_norm, index=i, total_sentences=len(clean_sentences))
            sentence_scores.append((i, sent, score))

        num_sentences = max(1, int(len(clean_sentences) * ratio))
        top_sentences = sorted(sentence_scores, key=lambda x: x[2], reverse=True)[:num_sentences]
        top_sentences.sort(key=lambda x: x[0])
        return "".join([item[1] for item in top_sentences]), sentence_scores


class TFIDFSimilarity_For_Comparison:
    def __init__(self):
        # 移除原本複雜的 regex，改用預設值 (因為我們會自己加空白)
        self.vectorizer = TfidfVectorizer()
        self.vocab = set()

    def fit(self, documents):
        # 先用 jieba 斷詞並加空白
        docs_cut = [" ".join(jieba.lcut(doc)) for doc in documents]
        self.tfidf_matrix = self.vectorizer.fit_transform(docs_cut)

    def get_similarity(self, text1, text2):
        # 對輸入的兩個文本進行斷詞
        t1_cut = " ".join(jieba.lcut(text1))
        t2_cut = " ".join(jieba.lcut(text2))

        # 計算相似度
        matrix = self.vectorizer.fit_transform([t1_cut, t2_cut])
        return cosine_similarity(matrix)[0][1]

# ==================== 3. 定義比較分析類別 (保留原始程式碼) ====================

class ComparisonAnalysis_With_Saving:

    def __init__(self):
        self.trad_sim = TFIDFSimilarity_For_Comparison()
        self.trad_clf = RuleBasedClassifier()
        self.trad_sum = StatisticalSummarizer()

        # 保持原始測試數據
        self.sim_data = [
            ("人工智慧是未來的趨勢", "AI技術將主導未來的發展"),
            ("今天天氣真好", "機器學習需要大量數據")
        ]
        self.clf_data = [
            "這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！",
            "最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好",
            "這部電影劇情空洞，演技糟糕，完全是浪費時間",
            "每天慢跑5公里，配合適當的重訓，體能進步很多"
        ]
        self.sum_article = """人工智慧 (AI) 的發展正在深刻改變我們的生活方式。從早上起床時的智慧鬧鐘，到通勤時的路線規劃，再到工作中的各種輔助工具，AI無處不在。\n\n在醫療領域，AI協助醫生進行疾病診斷，提高了診斷的準確率和效率。透過分析大量的醫療影像和病歷資料，AI能夠發現人眼容易忽略的細節，為患者提供更好的治療方案。\n\n教育方面，AI個人化學習系統能夠根據每個學生的學習進度和特點，提供客製化的教學內容。這種因材施教的方式，讓學習變得更加高效和有趣。\n\n然而，AI的快速發展也帶來了一些挑戰。首先是就業問題，許多傳統工作可能會被AI取代。其次是隱私和安全問題，AI系統需要大量數據來訓練，如何保護個人隱私成為重要議題。最後是倫理問題，AI的決策過程往往缺乏透明度，可能會產生偏見或歧視。\n\n面對這些挑戰，我們需要在推動AI發展的同時，建立相應的法律法規和倫理準則。只有這樣，才能確保AI技術真正為人類福祉服務，創造一個更美好的未來。"""

        self.results_dir = "results"
        os.makedirs(self.results_dir, exist_ok=True)
        self.performance_metrics = {}

    # 略過 plot_tfidf_similarity、compare_similarity、compare_classification 等函式的程式碼，
    # 這些函式將會正確調用 Section 1 和 Section 2 中定義的函式。

    def plot_tfidf_similarity(self, documents):
        print("\n[生成 TF-IDF 相似度矩陣圖...]")
        try:

            # 1. 先對所有文件進行中文斷詞，並用空白連接
            # 例如："人工智慧..." -> "人工智慧 正在 改變 世界..."
            documents_cut = [" ".join(jieba.lcut(doc)) for doc in documents]

            # 2. 使用預設的 TfidfVectorizer (它會依照空白切分)
            vectorizer = TfidfVectorizer()
            matrix = vectorizer.fit_transform(documents_cut)


            sim_matrix = cosine_similarity(matrix)
            doc_labels = [f"文件{i+1}" for i in range(len(documents))]

            plt.figure(figsize=(8, 6))
            # 使用支援中文的字型 (my_font 是您前面定義好的)
            ax = sns.heatmap(sim_matrix, annot=True, cmap="YlGnBu", fmt=".4f",
                             xticklabels=doc_labels, yticklabels=doc_labels)

            ax.set_xticklabels(doc_labels, fontproperties=my_font, rotation=0)
            ax.set_yticklabels(doc_labels, fontproperties=my_font, rotation=0)

            plt.title("TF-IDF 相似度矩陣 ", fontproperties=my_font, fontsize=14)
            plt.tight_layout()

            save_path = os.path.join(self.results_dir, "tfidf_similarity_matrix.png")
            plt.savefig(save_path)
            plt.close()
            print(f"✅ 圖片儲存成功 : {save_path}")
        except Exception as e:
            print(f"❌ 繪圖失敗: {e}")
            import traceback
            traceback.print_exc()

    def compare_similarity(self):
        times_trad, times_ai = [], []
        for t1, t2 in self.sim_data:
            s = time.time()
            self.trad_sim.get_similarity(t1, t2)
            times_trad.append(time.time() - s)
            s = time.time()
            ai_similarity(t1, t2)
            times_ai.append(time.time() - s)
        self.performance_metrics['sim_time_trad'] = np.mean(times_trad)
        self.performance_metrics['sim_time_ai'] = np.mean(times_ai)

    def compare_classification(self):
        times_trad, times_ai = [], []
        correct_trad, correct_ai = 0, 0
        expected_topics = ["美食", "科技", "電影", "運動"]

        results = []
        for i, text in enumerate(self.clf_data):
            # Traditional
            s = time.time()
            res_trad = self.trad_clf.classify(text)
            times_trad.append(time.time() - s)
            if res_trad['topic'] == expected_topics[i]: correct_trad += 1

            # AI
            s = time.time()
            res_ai = ai_classify(text)
            times_ai.append(time.time() - s)
            if expected_topics[i] in str(res_ai.get('topic', '')): correct_ai += 1

            results.append({
                "text": text,
                "trad": res_trad,
                "ai": res_ai
            })

        csv_path = os.path.join(self.results_dir, "classification_results.csv")
        pd.DataFrame(results).to_csv(csv_path, index=False, encoding='utf-8-sig')

        print(f"✅ 分類結果已儲存 (4筆資料): {csv_path}")


        self.performance_metrics['clf_time_trad'] = np.mean(times_trad)
        self.performance_metrics['clf_time_ai'] = np.mean(times_ai)
        self.performance_metrics['clf_acc_trad'] = (correct_trad / len(self.clf_data)) * 100
        self.performance_metrics['clf_acc_ai'] = (correct_ai / len(self.clf_data)) * 100

    def compare_summarization(self):
        # 這裡現在會呼叫 Section 1 中定義的真實 API 函式，而非模擬函式
        s = time.time()
        trad_res, _ = self.trad_sum.summarize(self.sum_article)
        time_trad = time.time() - s
        s = time.time()
        ai_res = ai_summarize(self.sum_article)
        time_ai = time.time() - s

        with open(os.path.join(self.results_dir, "summarization_comparison.txt"), "w", encoding="utf-8") as f:
            f.write(f"Traditional:\n{trad_res}\n\nAI:\n{ai_res}")

        self.performance_metrics['sum_time_trad'] = time_trad
        self.performance_metrics['sum_time_ai'] = time_ai

    def generate_comparison_table(self):
        print("\n[C-1 量化比較表格]")

        data = [
            ["相似度計算", "", "", ""],
            ["", "準確率", "低 (僅字面)", "高 (含語意)"],
            ["", "處理時間", f"{self.performance_metrics.get('sim_time_trad', 0):.4f}秒", f"{self.performance_metrics.get('sim_time_ai', 0):.4f}秒"],
            ["", "成本", "$0", "Token計費"],
            ["文本分類", "", "", ""],
            ["", "準確率", f"{self.performance_metrics.get('clf_acc_trad', 0):.0f}%", f"{self.performance_metrics.get('clf_acc_ai', 0):.0f}%"],
            ["", "處理時間", f"{self.performance_metrics.get('clf_time_trad', 0):.4f}秒", f"{self.performance_metrics.get('clf_time_ai', 0):.4f}秒"],
            ["", "支援類別", "有限", "無限"],
            ["自動摘要", "", "", ""],
            ["", "資訊保留度", "中 (抽取)", "高 (生成)"],
            ["", "語句通順度", "低 (拼接)", "高 (自然)"],
            ["", "長度控制", "困難", "容易"]
        ]

        columns = ["評估面向", "評估指標", "TF-IDF (傳統)", "GPT-4o (AI)"]
        df = pd.DataFrame(data, columns=columns)
        # 顯示 DataFrame
        display(df)
        #df.to_csv(os.path.join(self.results_dir, "quantitative_comparison.csv"), index=False, encoding='utf-8-sig')

    def save_metrics(self):
        with open(os.path.join(self.results_dir, "performance_metrics.json"), "w") as f:
            json.dump(self.performance_metrics, f, indent=4)

    def run_full_report(self):
        print("="*60)
        print("【NLP 技術對比分析報告】")
        print("="*60)

        self.compare_similarity()
        self.compare_classification()
        self.compare_summarization()
        self.plot_tfidf_similarity(documents_for_A1)
        self.generate_comparison_table()
        self.save_metrics()

        print("\n🎉 所有分析完成！")
        print(f"結果檔案位置: {self.results_dir}/")


# ==================== 4. 執行程式 ====================
analyzer = ComparisonAnalysis_With_Saving()
analyzer.run_full_report()

✅ 中文字型準備完成
✅ OpenAI Client 初始化完成
【NLP 技術對比分析報告】
✅ 分類結果已儲存 (4筆資料): results/classification_results.csv

[生成 TF-IDF 相似度矩陣圖...]


/usr/local/lib/python3.12/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 25991 (\N{CJK UNIFIED IDEOGRAPH-6587}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.12/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 20214 (\N{CJK UNIFIED IDEOGRAPH-4EF6}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()


✅ 圖片儲存成功 : results/tfidf_similarity_matrix.png

[C-1 量化比較表格]


,評估面向,評估指標,TF-IDF (傳統),GPT-4o (AI)
0,相似度計算,,,
1,,準確率,低 (僅字面),高 (含語意)
2,,處理時間,0.0055秒,0.6062秒
3,,成本,$0,Token計費
4,文本分類,,,
5,,準確率,50%,100%
6,,處理時間,0.0003秒,0.5522秒
7,,支援類別,有限,無限
8,自動摘要,,,
9,,資訊保留度,中 (抽取),高 (生成)



🎉 所有分析完成！
結果檔案位置: results/
